<img src="https://www3.um.edu.uy/logoum.jpg" width=300>
<h1 align="center">Thesis - KNN User User F1 </h1> 
<h2 align="center">Alejo Paullier</h2> 

https://lkpy.lenskit.org/en/stable/knn.html

In [4]:
from lenskit import batch, topn, util
from lenskit import crossfold as xf
from lenskit.algorithms import Recommender, user_knn as knn
from lenskit import topn
import pandas as pd
import numpy as np
np.seterr(divide='ignore', invalid='ignore')

ratings = pd.read_csv('C:\\Users\\Alejo\\Tesis\\Demo\\ml-100k\\u.data', sep='\t',
                      names=['user', 'item', 'rating', 'timestamp'])


def eval(aname, algo, train, test):
    fittable = util.clone(algo) # Object cloning means to create an exact copy of the original object.
    fittable = Recommender.adapt(fittable) 
    fittable.fit(train) # Entrenamos el algoritmo con el training dataset
    users = test.user.unique() # Devuelve un array con los users unicos en el testing dataset
    # now we run the recommender
    recs = batch.recommend(fittable, users, 100)
    # add the algorithm name for analyzability
    recs['Algorithm'] = aname
    return recs

In [2]:
def model_trainer(neighbours,center):
    neighbours=int(neighbours)
    all_recs = []
    test_data = []
    algo_uu = knn.UserUser(neighbours,center=center,aggregate='weighted-average')
    for train, test in xf.partition_users(ratings[['user', 'item', 'rating']], 5, xf.SampleFrac(0.2)):
        test_data.append(test)
        all_recs.append(eval('UserUser', algo_uu, train, test))
    all_recs = pd.concat(all_recs, ignore_index=True)
    test_data = pd.concat(test_data, ignore_index=True)
    rla1 = topn.RecListAnalysis()
    rla2 = topn.RecListAnalysis()
    rla1.add_metric(topn.precision)
    rla2.add_metric(topn.recall)
    prec = rla1.compute(all_recs, test_data)
    rec = rla2.compute(all_recs, test_data)
    F1 = ((prec['precision'].values*rec['recall'].values*2)/(prec['precision'].values+rec['recall'].values))
    F1 = np.nan_to_num(F1)
    return [F1.mean(),prec['precision'].mean(),rec['recall'].mean()]


In [5]:
from hyperopt import fmin, tpe, hp, STATUS_OK

def objective(params):
    neighbours = params['neighbours']
    center = params['center']
    metric = model_trainer(neighbours,center)
    print([neighbours,center,metric[0],metric[1],metric[2]],',')
    return {'loss': -metric[0], 'status': STATUS_OK }

space= {'neighbours': hp.uniform('neighbours', 1, 100),
       'center': hp.choice('center', ['True', 'False'])}

best = fmin(objective, space, algo=tpe.suggest,max_evals=100)

[42.89331093440601, 'True', 0.04554780109724322, 0.031071049840933285, 0.13744791557794114]                            
,                                                                                                                      
[72.90338538176424, 'False', 0.04237852432690795, 0.028939554612937533, 0.12680845761905818]                           
,                                                                                                                      
[83.57938263216137, 'False', 0.04148518851959778, 0.028205944798301557, 0.12736127481774412]                           
,                                                                                                                      
[64.59516423008958, 'True', 0.044456961807762346, 0.030339342523860106, 0.13288230828869105]                           
,                                                                                                                      
[51.027323265765965, 'True', 0.043368937

[38.962595710747784, 'False', 0.04531822078270244, 0.031028632025450743, 0.13419638949006835]                          
,                                                                                                                      
[48.565419626289696, 'True', 0.04508653806163979, 0.03080593849416762, 0.136363015609479]                              
,                                                                                                                      
[89.25225348620627, 'True', 0.042446360301361806, 0.028939554612937506, 0.12641132897141982]                           
,                                                                                                                      
[46.94014488756406, 'False', 0.043467881708576045, 0.029745493107105073, 0.1309220215463389]                           
,                                                                                                                      
[60.33125594733151, 'True', 0.0434912512

[18.365567174846806, 'False', 0.0446949350376626, 0.030816542948038265, 0.1297567231156366]                            
,                                                                                                                      
[22.181387808306866, 'False', 0.04812678642599629, 0.03287380699893963, 0.1443270389538889]                            
,                                                                                                                      
[3.9202563417807914, 'False', 0.04419581700772608, 0.03050901378579013, 0.12830455095277796]                           
,                                                                                                                      
[22.4451465278848, 'False', 0.04648980801854922, 0.0318557794273596, 0.13779384344968232]                              
,                                                                                                                      
[6.6357428933612255, 'False', 0.04656280

In [6]:
array = [[42.89331093440601, 'True', 0.04554780109724322, 0.031071049840933285, 0.13744791557794114]                            
,                                                                                                                      
[72.90338538176424, 'False', 0.04237852432690795, 0.028939554612937533, 0.12680845761905818]                           
,                                                                                                                      
[83.57938263216137, 'False', 0.04148518851959778, 0.028205944798301557, 0.12736127481774412]                           
,                                                                                                                      
[64.59516423008958, 'True', 0.044456961807762346, 0.030339342523860106, 0.13288230828869105]                           
,                                                                                                                      
[51.027323265765965, 'True', 0.04336893707582403, 0.02945917285259816, 0.13238317483695966]                            
,                                                                                                                      
[63.13411612327901, 'False', 0.042597464086057024, 0.02905620360551441, 0.12697467814882976]                           
,                                                                                                                      
[5.815284712143828, 'True', 0.04474515170417028, 0.030911983032873958, 0.13174551561867034]                            
,                                                                                                                      
[31.538723927864872, 'False', 0.045751760259392565, 0.03131495227995766, 0.13537946015363253]                          
,                                                                                                                      
[67.72531312508507, 'True', 0.04231817617963029, 0.028812301166490015, 0.12833482315475753]                            
,                                                                                                                      
[42.14580423351022, 'False', 0.045022277345274475, 0.030784729586426376, 0.13400813937884357]                          
,                                                                                                                      
[54.00143389270797, 'False', 0.04478614410449679, 0.030466595970307608, 0.13727269520168386]                           
,                                                                                                                      
[34.09788681866976, 'True', 0.04500915940612983, 0.03080593849416766, 0.1324405350422242]                              
,                                                                                                                      
[13.580591893352125, 'True', 0.04772989480375604, 0.03268292682926841, 0.14144259643919194]                            
,                                                                                                                      
[99.80000284420771, 'True', 0.040097120806456624, 0.027282377919320668, 0.12174719726069078]                           
,                                                                                                                      
[6.088816679992784, 'True', 0.0476140782219032, 0.03268292682926845, 0.1403648341384562]                               
,                                                                                                                      
[29.17311275209025, 'True', 0.044857122575652976, 0.030731707317073254, 0.13470096220764347]                           
,                                                                                                                      
[35.902860431640455, 'True', 0.04428749782901814, 0.03028632025450697, 0.13506281613058094]                            
,                                                                                                                      
[3.577986265681353, 'False', 0.04512613114793199, 0.030922587486744528, 0.1359084985775956]                            
,                                                                                                                      
[65.92508247582752, 'True', 0.04346462858982295, 0.029618239660657547, 0.1308385110257821]                             
,                                                                                                                      
[95.51948851967718, 'True', 0.042553981846627835, 0.028854718981972515, 0.13113540775866614]                           
,                                                                                                                      
[16.468849910643502, 'True', 0.046549578582552535, 0.0320678685047721, 0.13623856453725153]                            
,                                                                                                                      
[15.245046293813314, 'True', 0.04666575526416883, 0.03211028632025461, 0.13577849851615315]                            
,                                                                                                                      
[18.718768741446542, 'True', 0.046550098663020026, 0.03198303287380708, 0.1358763014095263]                            
,                                                                                                                      
[1.0343543935671757, 'True', 0.03883720367931145, 0.02663838812301173, 0.11918015897137223]                            
,                                                                                                                      
[23.71110290239157, 'True', 0.04548955714303274, 0.031166489925768894, 0.1348355918850667]                             
,                                                                                                                      
[9.326027323393154, 'True', 0.04647458608525449, 0.03195121951219521, 0.13709643681334982]                             
,                                                                                                                      
[23.013704962077867, 'True', 0.047116158116210984, 0.03226935312831396, 0.1411967944823923]                            
,                                                                                                                      
[12.273469886477416, 'True', 0.046388520949846745, 0.031876988335100866, 0.13399150491189554]                          
,                                                                                                                      
[42.42334495703934, 'True', 0.04355339564351307, 0.029745493107105076, 0.12950402954716417]                            
,                                                                                                                      
[1.2971178518720254, 'True', 0.0403369283705771, 0.0275053078556264, 0.12313398544384173]                              
,                                                                                                                      
[25.131926920359327, 'True', 0.04587112051886236, 0.031442205726405166, 0.13271609602897885]                           
,                                                                                                                      
[78.58995591171495, 'False', 0.04188419125483275, 0.02857900318133622, 0.12400926034458637]                            
,                                                                                                                      
[8.337383277294904, 'True', 0.04591273145871898, 0.031484623541887725, 0.13490398812345708]                            
,                                                                                                                      
[56.42763245196328, 'True', 0.04428810639436049, 0.030233297985153854, 0.13523608113731575]                            
,                                                                                                                      
[38.962595710747784, 'False', 0.04531822078270244, 0.031028632025450743, 0.13419638949006835]                          
,                                                                                                                      
[48.565419626289696, 'True', 0.04508653806163979, 0.03080593849416762, 0.136363015609479]                              
,                                                                                                                      
[89.25225348620627, 'True', 0.042446360301361806, 0.028939554612937506, 0.12641132897141982]                           
,                                                                                                                      
[46.94014488756406, 'False', 0.043467881708576045, 0.029745493107105073, 0.1309220215463389]                           
,                                                                                                                      
[60.33125594733151, 'True', 0.04349125125810467, 0.0295864262990457, 0.13167424882137935]                              
,                                                                                                                      
[19.4223759928408, 'False', 0.04569431294729773, 0.03127253446447516, 0.1373204225679617]                              
,                                                                                                                      
[28.605941732030814, 'True', 0.0451244080069221, 0.030880169671262007, 0.13474164532617608]                            
,                                                                                                                      
[72.29157820803779, 'True', 0.042554653041935966, 0.028939554612937495, 0.1320677993777969]                            
,                                                                                                                      
[13.280658184725613, 'False', 0.04737078558578566, 0.0324178154825028, 0.14329517884415788]                            
,                                                                                                                      
[10.336097212318158, 'True', 0.046549763479974235, 0.0319830328738071, 0.13894881463438388]                            
,                                                                                                                      
[4.914399110656868, 'True', 0.0473358291478605, 0.032460233297985246, 0.1422438279871469]                              
,                                                                                                                      
[32.150562312724865, 'False', 0.045972938700031656, 0.03139978791092272, 0.13795646486968874]                          
,                                                                                                                      
[38.23478001930844, 'True', 0.045743689358593, 0.031071049840933288, 0.13898708780557756]                              
,                                                                                                                      
[5.885112191136307, 'True', 0.04715891127376567, 0.032417815482502756, 0.13990539536164398]                            
,                                                                                                                      
[26.710157096209173, 'True', 0.04559336747451799, 0.03111346765641579, 0.1356839487712224]                             
,                                                                                                                      
[20.08451805335708, 'False', 0.04659595141609733, 0.03198303287380709, 0.13648187442256754]                            
,                                                                                                                      
[46.31130986147575, 'True', 0.044433275097141074, 0.030254506892895074, 0.1352108646273819]                            
,                                                                                                                      
[54.0892770181905, 'True', 0.043243985755207964, 0.02958642629904566, 0.12864074424254623]                             
,                                                                                                                      
[1.6161080311228755, 'True', 0.03850067567073906, 0.026256627783669176, 0.12136134679776622]                           
,                                                                                                                      
[34.507588334654976, 'False', 0.04450577641191814, 0.03059384941675512, 0.12965530951360763]                           
,                                                                                                                      
[15.651409284740259, 'True', 0.046724196359090864, 0.03208907741251338, 0.1368153311073306]                            
,                                                                                                                      
[30.763904210768775, 'True', 0.04656050608054872, 0.03181336161187711, 0.1406903446896245]                             
,                                                                                                                      
[22.196232902700196, 'True', 0.04661221688368179, 0.03196182396606583, 0.13755493937038238]                            
,                                                                                                                      
[84.90317742050965, 'True', 0.04195435687366416, 0.028441145281018137, 0.12973695713529204]                            
,                                                                                                                      
[39.3539094516393, 'True', 0.044456173941466795, 0.030381760339342617, 0.13249015717154175]                            
,                                                                                                                      
[7.762709001346908, 'False', 0.04630304384443011, 0.0318875927889715, 0.13531606877782637]                             
,                                                                                                                      
[60.26165002163516, 'True', 0.04458836378981495, 0.030402969247083852, 0.13461176970278446]                            
,                                                                                                                      
[12.40972755724522, 'True', 0.047302055323654696, 0.03241781548250276, 0.1440377623022534]                             
,                                                                                                                      
[1.192509373922821, 'True', 0.04129063317109749, 0.028112407211028704, 0.12731591031960562]                            
,                                                                                                                      
[98.57723970498832, 'False', 0.04246092116141931, 0.028812301166490036, 0.13355572916012307]                           
,                                                                                                                      
[71.00415930389525, 'True', 0.04168262939961045, 0.02840933191940624, 0.12638525209777957]                             
,                                                                                                                      
[13.331573173402742, 'False', 0.04600178549141464, 0.031686108165429584, 0.13343321474272998]                          
,                                                                                                                      
[16.124885305495464, 'False', 0.046079056588463736, 0.03171792152704146, 0.13515678328354294]                          
,                                                                                                                      
[26.599851426663356, 'False', 0.046802214675027515, 0.032046659597030874, 0.13849471415030778]                         
,                                                                                                                      
[18.365567174846806, 'False', 0.0446949350376626, 0.030816542948038265, 0.1297567231156366]                            
,                                                                                                                      
[22.181387808306866, 'False', 0.04812678642599629, 0.03287380699893963, 0.1443270389538889]                            
,                                                                                                                      
[3.9202563417807914, 'False', 0.04419581700772608, 0.03050901378579013, 0.12830455095277796]                           
,                                                                                                                      
[22.4451465278848, 'False', 0.04648980801854922, 0.0318557794273596, 0.13779384344968232]                              
,                                                                                                                      
[6.6357428933612255, 'False', 0.04656280022069577, 0.032120890774125235, 0.1356677129726569]                           
,                                                                                                                      
[9.213831157203005, 'False', 0.04695788631495037, 0.03233297985153774, 0.13504322912005032]                            
,                                                                                                                      
[36.606959246821255, 'False', 0.04485332386062547, 0.030625662778367025, 0.13715787289035597]                          
,                                                                                                                      
[10.597812921104795, 'False', 0.04699868188489102, 0.032343584305408395, 0.13766191510988038]                          
,                                                                                                                      
[42.12096233619353, 'True', 0.04438359392284155, 0.030318133616118882, 0.1327226598731393]                             
,                                                                                                                      
[28.988631695826996, 'False', 0.045933771343731514, 0.03136797454931077, 0.13729423607505642]                          
,                                                                                                                      
[50.14928222681206, 'True', 0.043599477138335264, 0.02986214209968198, 0.1292917278221366]                             
,                                                                                                                      
[20.66032996299054, 'True', 0.0460621469046126, 0.03147401908801707, 0.13781433526124437]                              
,                                                                                                                      
[32.80221741091152, 'False', 0.04377505041130442, 0.029957582184517557, 0.13297272518854983]                           
,                                                                                                                      
[25.167495423222334, 'True', 0.04403361022261699, 0.030190880169671364, 0.1290313750015424]                            
,                                                                                                                      
[17.799198879195725, 'True', 0.04673813157617022, 0.032099681866383976, 0.13608141270827626]                           
,                                                                                                                      
[11.370408341160697, 'False', 0.04652625938015248, 0.032014846235419, 0.13678809344127432]                             
,                                                                                                                      
[44.529230308975656, 'True', 0.04250513180833136, 0.029141039236479396, 0.12558424870255877]                           
,                                                                                                                      
[14.991189136297919, 'True', 0.04723884246442599, 0.032269353128314006, 0.14013056383490335]                           
,                                                                                                                      
[2.864060005675576, 'False', 0.04239530163942943, 0.029225053078556343, 0.12526659596167414]                           
,                                                                                                                      
[40.35464637622863, 'True', 0.04474297024416861, 0.030625662778366966, 0.12879155823712313]                            
,                                                                                                                      
[36.61963779407917, 'True', 0.04467695802978132, 0.030562036055143234, 0.13180542238768347]                            
,                                                                                                                      
[24.824457734589263, 'False', 0.04666069367683976, 0.03199363732767774, 0.1373629465018214]                            
,                                                                                                                      
[5.501582805749636, 'True', 0.046350385781122785, 0.03183457051961837, 0.1386646630551595]                             
,                                                                                                                      
[30.20966789525024, 'True', 0.044716163531167204, 0.030604453870625773, 0.1337014028221426]                            
,                                                                                                                      
[80.0437192228751, 'False', 0.04210009339250874, 0.02872746553552498, 0.12552467791359745]                             
,                                                                                                                      
[27.98749226707498, 'True', 0.04623761819276226, 0.03169671261930019, 0.13628398236584507]                             
,                                                                                                                      
[56.088059573440695, 'True', 0.04440118976361314, 0.030275715800636344, 0.13398246243977305]                           
,                                                                                                                      
[14.17341635278464, 'False', 0.04752662323551942, 0.03283138918345717, 0.13630655990149285]                            
,                                                                                                                      
[7.849723842620589, 'True', 0.04553436629189207, 0.03142099681866396, 0.1331368205946202]                              
,                                                                                                                      
[34.073082308947384, 'True', 0.04484511220847371, 0.03059384941675514, 0.13372068527073103]                            
,                                                                                                                      
[17.307403371294804, 'False', 0.0468785742778211, 0.031961823966065804, 0.14175537161257581]                           
,                                                                                                                      
[61.324943349062806, 'True', 0.04437130090668183, 0.030201484623541955, 0.13611333776955029]  ]

In [7]:
metric = pd.DataFrame(array)
metric.columns = ['neighbours','center','F1','Precision','Recall']
metric.head(5)

,neighbours,center,F1,Precision,Recall
0,42.893311,True,0.045548,0.031071,0.137448
1,72.903385,False,0.042379,0.028940,0.126808
2,83.579383,False,0.041485,0.028206,0.127361
3,64.595164,True,0.044457,0.030339,0.132882
4,51.027323,True,0.043369,0.029459,0.132383


In [8]:
metric.sort_values(by=['F1'],ascending=False).head(10)

,neighbours,center,F1,Precision,Recall
69,22.181388,False,0.048127,0.032874,0.144327
12,13.580592,True,0.047730,0.032683,0.141443
14,6.088817,True,0.047614,0.032683,0.140365
95,14.173416,False,0.047527,0.032831,0.136307
42,13.280658,False,0.047371,0.032418,0.143295
44,4.914399,True,0.047336,0.032460,0.142244
61,12.409728,True,0.047302,0.032418,0.144038
85,14.991189,True,0.047239,0.032269,0.140131
47,5.885112,True,0.047159,0.032418,0.139905
26,23.013705,True,0.047116,0.032269,0.141197


In [9]:
metric.to_csv('metric_F1.csv', index=False)